# MGIMO intensive

## Make AI work for you

### 1. Libraries

In [ ]:
import io
import re
import json
import pandas as pd
from typing import Tuple, Optional, Any

from yandexgpt_client import YandexGPTClient

### 2. Utilities functions

In [ ]:
def json_data(file_path):
    """Load and return JSON data from a file.
    
    Args:
        file_path (str): Path to the JSON file.
    
    Returns:
        dict or list: Parsed JSON data.
    """
    with open(file_path) as file:
        data = json.load(file)
    return data


def capture_dataframe_info(df: pd.DataFrame) -> str:
    """
    Capture the output of df.info() into a string variable.

    Args:
        df: DataFrame to get info from.

    Returns:
        String containing the formatted info output.
    """
    # Create a StringIO object to capture the output
    buffer = io.StringIO()
    
    # Redirect stdout to our buffer temporarily
    df.info(buf=buffer)
    
    # Get the string value from the buffer
    info_string = buffer.getvalue()
    
    # Close the buffer
    buffer.close()
    
    return info_string


def extract_python_code(text: str) -> Optional[str]:
    """
    Extract Python code from markdown text containing code blocks.

    Args:
        markdown_text: String containing markdown with Python code blocks.

    Returns:
        Extracted Python code as a string, or None if no code blocks found.
    """
    pattern = r"```python\s*(.*?)\s*```"
    matches = re.findall(pattern, text, re.DOTALL)
    
    if matches:
        return "\n".join(match.strip() for match in matches)
    
    return None

### 3. Load data to analyse

In [ ]:
file_path = "../2_data/data_section35_112_v20250918_section_file/data_section35_112_v20250918.csv"

df = pd.read_csv(file_path, sep=";")
df.info()

### 4. Do some analysis

#### 4.1. Copypaste approach

Use the "old-school" approach and create prompt:
```prompt
Напиши код на Python для определения списка уникальных регионов в Pandas датафрейме. 

---
Описание датафрейма:
<ВЫВОД МЕТОДА df.info()>
```

In [ ]:
# Получаем уникальные значения из столбца region_name
unique_regions = df['region_name'].unique()

# Преобразуем в список (если нужно)
unique_regions_list = unique_regions.tolist()

print(f"Количество уникальных регионов: {len(unique_regions_list)}")
print("Список уникальных регионов:")
for region in unique_regions_list:
    print(region)

#### 4.2. Smart way

#### We need AI assistent

In [ ]:
# Load credentials for YandexGPT access
access_data = json_data('/home/jovyan/__DATA/mgimo_intensive/.accessyagpt')

In [ ]:
instruction = """
Вы — ИИ-ассистент для анализа данных.
Ваша задача — разрабатывать код на Python для анализа таблиц данных.
Получите запрос от пользователя и напишите код для ответа на вопрос пользователя.
Используйте информацию о структуре DataFrame библиотеки Pandas.
Всегда красиво выводите результат анализа с дополнительными комментариями.
Предоставляйте код Python, отмеченный '```python' в начале 
и '```' в конце для удобного извлечения.

Всегда обращайтесь к пользователю «Мой повелитель» и будьте очень вежливы.

Если проблему невозможно решить с предоставленными данными, 
попросите пользователя предоставить больше информации.
"""

yaclient = YandexGPTClient(
    folder_id=access_data["folder_id"],
    api_key=access_data["api_key"],
    instruction_text=instruction
)

#### We need to formulate a question

In [ ]:
# Extract info about dataframe

df_info = capture_dataframe_info(df)
print(df_info)

In [ ]:
# Make a proper prompt

question = "Сколько уникальных регионов в Pandas датафрейме?"

context_text = f"Dataframe info: {df_info}"

prompt = f"""
Возьмите вопрос и предоставьте код на Python 
для извлечения данных из датафрейма. Используй 
информацию о датафрейме из контекста.

ВОПРОС: {question}

КОНТЕКСТ: {context_text}
"""

print(prompt)

In [ ]:
# Ask our AI assistant

response = yaclient.call_yandexgpt(
    prompt=prompt, 
    model_name="yandexgpt", 
    max_tokens=1000, 
    temperature=0.1
)

In [ ]:
# See what is the answer

print(response)

#### How to run code?

In [ ]:
code_to_run = extract_python_code(response)
exec(code_to_run)

### 5. Test AI assistant

In [ ]:
def ai_assist(
    question: str, 
    df: pd.DataFrame, 
    yaclient: Any
) -> None:
    """Generates and executes Python code to query 
    DataFrame based on natural language question.
    
    Uses YandexGPT to convert natural language question 
    into Python code for DataFrame manipulation.
    The generated code is extracted and executed in the current context.
    
    Args:
        question: Natural language question about the DataFrame
        df: Input DataFrame to query
        yaclient: YandexGPT API client instance
    
    Returns:
        None
    """
    context_text = f"Dataframe info: {df_info}"

    prompt = f"""
    Возьмите вопрос и предоставьте код на Python 
    для извлечения данных из датафрейма. Используй 
    информацию о датафрейме из контекста.
    
    ВОПРОС: {question}
    
    КОНТЕКСТ: {context_text}
    """

    response = yaclient.call_yandexgpt(
        prompt=prompt, 
        model_name="yandexgpt", 
        max_tokens=1000, 
        temperature=0.1
    )

    code_to_run = extract_python_code(response)
    exec(code_to_run)

    return None

In [ ]:
question = "Сколько уникальных регионов в Pandas датафрейме?"

ai_assist(
    question=question, 
    df=df, 
    yaclient=yaclient
)

In [ ]:
question = "Выведи уникальные регионы"

ai_assist(
    question=question, 
    df=df, 
    yaclient=yaclient
)

In [ ]:
question = "Выведи среднее по индикаторам"

ai_assist(
    question=question, 
    df=df, 
    yaclient=yaclient
)

In [ ]:
question = "Выведи все уникальные индикаторы"

ai_assist(
    question=question, 
    df=df, 
    yaclient=yaclient
)

In [ ]:
question = """
Выведи гистограмму по годам для индикатора 
`Количество объектов, имеющих стационарные источники загрязнения атмосферного воздуха`
"""

ai_assist(
    question=question, 
    df=df, 
    yaclient=yaclient
)